ETL (Extract Transform Load) Script for getting the data. We first scrape the data using bs4. We can use pandas, since its a one-off, small-size script. Afterwards we get the election data and process them and merge them together into a single csv.
### TODO how did we get the gemeinden list?
### todo is this webscaping script good?

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Base URL
base_url = 'https://vrv97.offenerhaushalt.at/gemeinde/{}/finanzdaten/hauptansicht/unterricht-erziehung-sport-und-wissenschaft/absolut/{}/ausgaben'

# List of municipalities
gemeinden = [
   "adlwang", "aichkirchen", "aigen-schlägl", "aistersheim", "alberndorf-in-der-riedmark",
    "alkoven", "allerheiligen-im-mühlkreis", "allhaming", "altenberg-bei-linz", "altenfelden",
    "altheim", "altmünster", "altschwendt", "ampflwang-im-hausruckwald", "andorf", "andrichsfurt",
    "ansfelden", "antiesenhofen", "arbing", "arnreit", "aschach-an-der-donau", "aschach-an-der-steyr",
    "aspach", "asten", "attersee-am-attersee", "attnang-puchheim", "atzbach", "atzesberg", 
    "auberg", "auerbach", "aurach-am-hongar", "aurolzmünster", "bachmanning", 
    "bad-goisern-am-hallstättersee", "bad-hall", "bad-ischl", "bad-kreuzen", "bad-leonfelden", 
    "bad-schallerbach", "bad-wimsbach-neydharting", "bad-zell", "baumgartenberg", 
    "berg-im-attergau", "braunau-am-inn", "brunnenthal", "buchkirchen", "burgkirchen", 
    "desselbrunn", "diersbach", "dietach", "dimbach", "dorf-an-der-pram", "ebensee-am-traunsee", 
    "eberschwang", "eberstalzell", "edlbach", "edt-bei-lambach", "eferding", "eggelsberg", 
    "eggendorf-im-traunkreis", "eggerding", "eidenberg", "eitzing", "engelhartszell-an-der-donau", 
    "engerwitzdorf", "enns", "enzenkirchen", "eschenau-im-hausruckkreis", "esternberg", 
    "feldkirchen-an-der-donau", "feldkirchen-bei-mattighofen", "fischlham", "fornach", 
    "fraham", "frankenburg-am-hausruck", "frankenmarkt", "franking", "freinberg", 
    "freistadt", "gaflenz", "gallneukirchen", "gallspach", "gampern", "garsten", 
    "gaspoltshofen", "geboltskirchen", "geiersberg", "geinberg", "geretsberg", 
    "gilgenberg-am-weilhart", "gmunden", "goldwörth", "gosau", "gramastetten", "grein", 
    "grieskirchen", "großraming", "grünau-im-almtal", "grünbach", "grünburg", "gschwandt", 
    "gunskirchen", "gurten", "gutau", "haag-am-hausruck", "hagenberg-im-mühlkreis", 
    "haibach-im-mühlkreis", "haibach-ob-der-donau", "haigermoos", "hallstatt", "handenberg", 
    "hargelsberg", "hartkirchen", "haslach-an-der-mühl", "heiligenberg", "helfenberg", 
    "hellmonsödt", "helpfau-uttendorf", "herzogsdorf", "hinterstoder", "hinzenbach", 
    "hirschbach-im-mühlkreis", "hochburg-ach", "hofkirchen-an-der-trattnach", 
    "hofkirchen-im-mühlkreis", "hofkirchen-im-traunkreis", "hohenzell", "höhnhart", 
    "holzhausen", "hörbich", "hörsching", "innerschwand-am-mondsee", "inzersdorf-im-kremstal", 
    "jeging", "julbach","kallham", "kaltenberg", "katsdorf", "kefermarkt", "kematen-am-innbach", "kematen-an-der-krems", 
"kirchberg-bei-mattighofen", "kirchberg-ob-der-donau", "kirchberg-thening", "kirchdorf-am-inn", 
"kirchdorf-an-der-krems", "kirchham", "kirchheim-im-innkreis", "kirchschlag-bei-linz", 
"klaffer-am-hochficht", "klam", "klaus-an-der-pyhrnbahn", "kleinzell-im-mühlkreis", 
"kollerschlag", "königswiesen", "kopfing-im-innkreis", "kremsmünster", "krenglbach", 
"kronstorf", "laakirchen", "lambach", "lambrechten", "langenstein", "lasberg", 
"laussa", "lembach-im-mühlkreis", "lengau", "lenzing-an-der-ager", "leonding", 
"leopoldschlag", "lichtenau-im-mühlkreis", "lichtenberg", "liebenau", "linz", 
"lochen-am-see", "lohnsburg-am-kobernaußerwald", "losenstein", "luftenberg-an-der-donau", 
"manning", "marchtrenk", "maria-neustift", "maria-schmolln", "mattighofen", 
"mauerkirchen", "mauthausen", "mayrhof", "meggenhofen", "mehrnbach", "mettmach", 
"michaelnbach", "micheldorf-in-oberösterreich", "mining", "mitterkirchen-im-machland", 
"molln", "mondsee", "moosbach", "moosdorf", "mörschwang", "mühlheim-am-inn", 
"munderfing", "münzbach", "münzkirchen" "naarn-im-machlande", "natternbach", "nebelberg", "neufelden", "neuhofen-an-der-krems", 
    "neuhofen-im-innkreis", "neukirchen-am-walde", "neukirchen-an-der-enknach", 
    "neukirchen-an-der-vöckla", "neukirchen-bei-lambach", "neumarkt-im-hausruckkreis", 
    "neumarkt-im-mühlkreis", "neustift-im-mühlkreis", "niederkappel", "niederneukirchen", 
    "niederthalheim", "niederwaldkirchen", "nussbach", "nussdorf-am-attersee", 
    "oberhofen-am-irrsee", "oberkappel", "obernberg-am-inn", "oberndorf-bei-schwanenstadt", 
    "oberneukirchen", "oberschlierbach", "obertraun", "oberwang", "oepping", "offenhausen", 
    "oftering", "ohlsdorf", "ort-im-innkreis", "ostermiething", "ottenschlag-im-mühlkreis", 
    "ottensheim", "ottnang-am-hausruck", "pabneukirchen", "palting", "pasching", "pattigham", 
    "peilstein-im-mühlviertel", "pennewang", "perg", "perwang-am-grabensee", "peterskirchen", 
    "pettenbach", "peuerbach", "pfaffing", "pfaffstätt", "pfarrkirchen-bei-bad-hall", 
    "pfarrkirchen-im-mühlkreis", "piberbach", "pichl-bei-wels", "pierbach", "pilsbach", 
    "pinsdorf", "pischelsdorf-am-engelbach", "pitzenberg", "pollham", "polling-im-innkreis", 
    "pöndorf", "pötting", "pram", "prambachkirchen", "pramet", "pregarten", "puchenau", 
    "puchkirchen-am-trattberg", "pucking", "pühret", "pupping", "putzleinsdorf", "raab", 
    "rainbach-im-innkreis", "rainbach-im-mühlkreis", "rechberg", "redleiten", "redlham", 
    "regau", "reichenau-im-mühlkreis", "reichenthal", "reichersberg", "reichraming", 
    "ried-im-innkreis", "ried-im-traunkreis", "ried-in-der-riedmark", "riedau", 
    "rohr-im-kremstal", "rohrbach-berg", "roitham-am-traunfall", "rosenau-am-hengstpass", 
    "rossbach", "rossleithen", "rottenbach", "rüstorf", "rutzenham", "sandl", "sarleinsbach", 
    "sattledt", "saxen", "schalchen""schardenberg", "schärding", "scharnstein", "scharten", "schenkenfelden", "schiedlberg", "schildorn", "schlatt", "schleißheim", "schlierbach", "schlüsslberg", "schönau-im-mühlkreis", "schörfling-am-attersee", "schwand-im-innkreis", "schwanenstadt", "schwarzenberg-am-böhmerwald", "schwertberg", "seewalchen-am-attersee", "senftenbach", "sierning", "sigharting", "sipbachzell", "sonnberg-im-mühlkreis", "spital-am-pyhrn", "st-aegidi", "st-agatha", "st-florian", "st-florian-am-inn", "st-georgen-am-fillmannsbach", "st-georgen-am-walde", "st-georgen-an-der-gusen", "st-georgen-bei-grieskirchen", "st-georgen-bei-obernberg-am-inn", "st-georgen-im-attergau", "st-gotthard-im-mühlkreis", "st-johann-am-walde", "st-johann-am-wimberg", "st-konrad", "st-leonhard-bei-freistadt", "st-lorenz", "st-marien", "st-marienkirchen-am-hausruck", "st-marienkirchen-an-der-polsenz", "st-marienkirchen-bei-schärding", "st-martin-im-innkreis", "st-martin-im-mühlkreis", "st-nikola-an-der-donau", "st-oswald-bei-freistadt", "st-oswald-bei-haslach", "st-pankraz", "st-pantaleon", "st-peter-am-hart", "st-peter-am-wimberg", "st-radegund", "st-roman", "st-stefan-afiesl", "st-thomas", "st-thomas-am-blasenstein", "st-ulrich-bei-steyr", "st-ulrich-im-mühlkreis", "st-veit-im-innkreis", "st-veit-im-mühlkreis", "st-willibald", "st-wolfgang-im-salzkammergut", "stadl-paura", "steegen", "steinbach-am-attersee", "steinbach-am-ziehberg", "steinbach-an-der-steyr", "steinerkirchen-an-der-traun", "steinhaus", "steyr", "steyregg", "straß-im-attergau", "stroheim", "suben", "taiskirchen-im-innkreis", "tarsdorf", "taufkirchen-an-der-pram", "taufkirchen-an-der-trattnach", "ternberg", "thalheim-bei-wels", "tiefgraben", "timelkam", "tollet", "tragwein", "traun", "traunkirchen", "treubach", "tumeltsham", "überackern", "ulrichsberg", "ungenach", "unterach-am-attersee", "unterweißenbach", "unterweitersdorf", "utzenaich", "vichtenstein", "vöcklabruck", "vöcklamarkt", "vorchdorf", "vorderstoder", "vorderweißenbach", "waizenkirchen", "waldburg", "waldhausen-im-strudengau", "walding", "waldkirchen-am-wesen", "waldneukirchen", "waldzell", "wallern-an-der-trattnach", "wartberg-an-der-krems", "wartberg-ob-der-aist", "weibern", "weilbach", "weißenkirchen-im-attergau", "weißkirchen-an-der-traun", "weitersfelden", "wels", "wendling", "weng-im-innkreis", "wernstein-am-inn", "weyer", "weyregg-am-attersee", "wilhering", "windhaag-bei-freistadt", "windhaag-bei-perg", "windischgarsten", "wippenham", "wolfern", "wolfsegg-am-hausruck", "zell-am-moos", "zell-am-pettenfirst", "zell-an-der-pram", "zwettl-an-der-rod"
]

all_data = []

# Loop through each municipality
for gemeinde in gemeinden:
    # Loop through the years from 2007 to 2019
    for year in range(2007, 2020):
        # Construct URL
        url = base_url.format(gemeinde, year)
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Find the specific div by ID
            treemap_div_id = f'treemap-table-{year}'
            treemap_div = soup.find('div', id=treemap_div_id)
            if treemap_div:
                # Find the table within the div
                table = treemap_div.find('table')
                if table:
                    # Iterate through each row in the table body
                    for row in table.find_all('tr')[1:]:  # Skip the header row
                        columns = row.find_all('td')
                        if len(columns) >= 3:  # Ensure there are enough columns
                            section = columns[1].get_text(strip=True)  # "Abschnitt"
                            amount = columns[2].find('span', class_='value-value').get_text(strip=True)  # "Betrag in Euro"
                            all_data.append({'Gemeinde': gemeinde, 'Year': year, 'Abschnitt': section, 'Betrag in Euro': amount})
                else:
                    print(f"No data table found for {gemeinde} in year {year}.")
            else:
                print(f"No data available for {gemeinde} in year {year}. The div with ID 'treemap-table-2019' was not found.")
        else:
            print(f"Failed to retrieve data for {gemeinde} in year {year}. Status code: {response.status_code}")

df = pd.DataFrame(all_data)
df.to_csv('Bildungsausgaben_Gemeinden_Oberösterreich_data_2007_bis_2019(1).csv', index=False)
print("Data has been scraped and saved to 'Bildungsausgaben_Gemeinden_Oberösterreich_data_2007_bis_2019.csv'.")

KeyboardInterrupt: 

Afterwards, we get the data for the elections. We will use spark from now on, to ensure scalability.

### TODO how did we get election data

Since the election data was originally provided in an inconsistent XLS format—with irregular merged cells, random empty lines, and other structural issues—we manually formatted the data to ensure consistency and saved it as an XLSX file for better usability.

Since we had only 5 XLS files for election data, this was the easiest option. However, this can be automated using a script for formatting if we were to automate and scale this project. However, this is out of scope for now.

Afterwards, we used a small script to convert these XLSX files to CSV.

In [38]:
import os
import pandas as pd

data_dir = os.path.join(os.getcwd(), "data")

# Get all .xlsx files in the data directory
xlsx_files = [f for f in os.listdir(data_dir) if f.endswith(".xlsx")]

for file in xlsx_files:
    file_path = os.path.join(data_dir, file)
    try:
        df = pd.read_excel(file_path, sheet_name=None)
        for sheet_name, sheet_df in df.items():
            # Replace spaces with underscores in filename
            sanitized_filename = os.path.splitext(file)[0].replace(" ", "_")
            sanitized_sheet_name = sheet_name.replace(" ", "_")
            csv_filename = os.path.join(data_dir, f"{sanitized_filename}_{sanitized_sheet_name}.csv")

            sheet_df.to_csv(csv_filename, index=False, sep=",", encoding="latin1")
            print(f"Converted: {file} (Sheet: {sheet_name}) -> {csv_filename}")
    except Exception as e:
        print(f"Error converting {file}: {e}")


Converted: OÖ 2017.xlsx (Sheet: Stimmen) -> /home/jovyan/DP2/Project/data/OÖ_2017_Stimmen.csv
Converted: OÖ 2017.xlsx (Sheet: Mandate) -> /home/jovyan/DP2/Project/data/OÖ_2017_Mandate.csv
Converted: OÖ 2019.xlsx (Sheet: Stimmen) -> /home/jovyan/DP2/Project/data/OÖ_2019_Stimmen.csv
Converted: OÖ 2019.xlsx (Sheet: Mandate) -> /home/jovyan/DP2/Project/data/OÖ_2019_Mandate.csv
Converted: OÖ 2024.xlsx (Sheet: Stimmen) -> /home/jovyan/DP2/Project/data/OÖ_2024_Stimmen.csv
Converted: OÖ 2024.xlsx (Sheet: Mandate) -> /home/jovyan/DP2/Project/data/OÖ_2024_Mandate.csv
Converted: OÖ 2008.xlsx (Sheet: Stimmen) -> /home/jovyan/DP2/Project/data/OÖ_2008_Stimmen.csv
Converted: OÖ 2008.xlsx (Sheet: Mandate) -> /home/jovyan/DP2/Project/data/OÖ_2008_Mandate.csv
Converted: OÖ 2013.xlsx (Sheet: Stimmen) -> /home/jovyan/DP2/Project/data/OÖ_2013_Stimmen.csv
Converted: OÖ 2013.xlsx (Sheet: Mandate) -> /home/jovyan/DP2/Project/data/OÖ_2013_Mandate.csv
Converted: OÖ 2013.xlsx (Sheet: whlsortgemnr) -> /home/jovya

OÖ 2013.xlsx contained 3 (hidden) sheets, which upon manual inspection, are not relevant and/or redundant for us. Therefore we remove them.

In [30]:
!rm data/OÖ_2013_whlsortgemnr.csv
!rm data/OÖ_2013_MandateData.csv
!rm data/OÖ_2013_WAHLDAT.csv

In [39]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, when, lit, expr
import pyspark.sql.functions as F
import os
from unidecode import unidecode
# init spark
spark = (
    SparkSession.builder
    .appName("MunicipalSpendingAndElectionAnalysis")
    .getOrCreate()
)

Then we import the data from the CSV files and clean them. We start with the spendings data:
- we remove encoding issues we noticed upon manual inspection
- we rename categories for readability and cast their type
- we aggregate the spending data.
    - we do this becuase the data is currently broken down into subcategories (e.g. spendings on preschool education, on physical education etc.)
    - for some of the analysis we will use the total spending per municipality
- we slugify using the helper function the municipality names so we can process them easier

In [36]:
# helper function
def standardize_municipality(col_name):
    """lowercase, trim, replace spaces with dashes (i.e. slugify)"""
    return F.regexp_replace(F.lower(F.trim(F.col(col_name))), " ", "-")


# --- read & transform municipal spending data ---
spending_csv = "data/Bildungsausgaben_Gemeinden_Oberösterreich_data_2007_bis_2019.csv"
df_spending = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .option("encoding", "UTF-8")
    .csv(spending_csv)
)

# rename cols
df_spending = (df_spending
    .withColumnRenamed("Gemeinde", "Municipality")
    .withColumnRenamed("Year", "Year")
    .withColumnRenamed("Abschnitt", "Category")
    .withColumnRenamed("Betrag in Euro", "Spending")
)

# replace chars in category
df_spending = df_spending.withColumn(
    "Category",
    regexp_replace(col("Category"), "FĂ¶rderung", "Förderung")
)
df_spending = df_spending.withColumn(
    "Category",
    regexp_replace(col("Category"), "Sport und auĂźerschulische Leibeserziehung",
                   "Sport und außerschulische Leibeserziehung")
)

# cast cols
df_spending = df_spending.withColumn("Year", col("Year").cast("int"))
df_spending = df_spending.withColumn("Spending", col("Spending").cast("float"))

# aggregate spending
df_spending_agg = (
    df_spending
    .groupBy("Municipality", "Year")
    .agg(F.sum("Spending").alias("Spending_Summe"))
)

# add lowercase municipality
df_spending_agg = df_spending_agg.withColumn("Municipality_Lowercase", standardize_municipality("Municipality"))
print("Done")

Done


After this, we process the election data for the available years. We initialize an empty DataFrame with predefined columns to store the cleaned data. The predefined columns are consistent amongst Austrian election datasets. Therefore, this script would presumably work for any election data across Austria. For including other countries, at first, we would need to standardize their election data to this format.

We read the CSV into Spark, standardize column names, and identify party vote columns. We convert party vote counts to integers and determine the winning party for each municipality, which we will use for predictive analysis later on. We also add a slugified version of the municipality name for consistency.

Once cleaned, the election data is merged into a single DataFrame and joined with the spending data using slugified municipality name and year. Since spending data only covers 2007–2019, elections from 2024 are automatically excluded. However, upon adding recent spending data, 2024 would be automatically included without needing manual modification to this script. Finally, we save the merged dataset as a CSV.

In [37]:

# --- read & transform election data ---
election_yrs = [2008, 2013, 2017, 2019, 2024]
all_election_df = spark.createDataFrame([], schema="Year INT, Municipality_ID STRING, Municipality_Name STRING, Wahlberechtigte INT, abgegebene_Stimmen INT, gueltige_Stimmen INT, Wahlbeteiligung STRING, Winning_Party STRING, Municipality_Lowercase STRING")

for year in election_yrs:
    print(f"Processing data for {year}")
    votes_file = f"data/OÖ_{year}_Stimmen.csv"

    # skip if file missing
    if not os.path.exists(votes_file):
        print(f"[WARNING] Missing CSV(s) for year {year} -> Skipping.")
        continue

    # read csv
    votes_df = (
        spark.read
        .option("header", "true")
        .option("inferSchema", "true")
        .option("encoding", "latin1")
        .csv(votes_file)
    )

    # normalize col names
    renamed_columns = {c: unidecode(c.strip()).replace(" ", "_") for c in votes_df.columns}
    votes_df = votes_df.selectExpr(*[f"`{old}` as `{new}`" for old, new in renamed_columns.items()])
    
    # identify party cols
    party_columns = [c for c in votes_df.columns if c not in {
        "Nr.", "Name", "Wahlberechtigte", "abgegeb._Stimmen", "gultige", "Wahlbet.", "ungultige"
    }]

    # rename cols
    votes_df = votes_df.withColumnRenamed("Nr.", "Municipality_ID")
    votes_df = votes_df.withColumnRenamed("Name", "Municipality_Name")
    votes_df = votes_df.withColumnRenamed("Wahlberechtigte", "Wahlberechtigte")
    votes_df = votes_df.withColumnRenamed("abgegeb._Stimmen", "abgegebene_Stimmen")
    votes_df = votes_df.withColumnRenamed("gultige", "gueltige_Stimmen")
    votes_df = votes_df.withColumnRenamed("Wahlbet.", "Wahlbeteiligung")

    # cast party cols to int
    for party_col in party_columns:
        votes_df = votes_df.withColumn(f"votes_{party_col}", col(party_col).cast("int"))

    # determine win party
    vote_cols = [F.coalesce(col(f"votes_{party_col}"), lit(0)) for party_col in party_columns]
    votes_df = votes_df.withColumn("max_votes", F.greatest(*vote_cols))

    winning_party_cases = [when(col("max_votes") == col(f"votes_{party_col}"), lit(party_col)) for party_col in party_columns]
    votes_df = votes_df.withColumn("Winning_Party", F.coalesce(*winning_party_cases))

    # drop temp cols
    for party_col in party_columns:
        votes_df = votes_df.drop(f"votes_{party_col}")

    votes_df = votes_df.drop("max_votes")

    # add year & municipality lowercase
    votes_df = votes_df.withColumn("Year", lit(year))
    votes_df = votes_df.withColumn("Municipality_Lowercase", standardize_municipality("Municipality_Name"))
    # filter for numeric municipality ids
    votes_df = votes_df.filter(col("Municipality_ID").rlike("^[0-9]+$"))

    # add to main df
    all_election_df = all_election_df.unionByName(votes_df.select(
        "Year", "Municipality_ID", "Municipality_Name", "Wahlberechtigte",
        "abgegebene_Stimmen", "gueltige_Stimmen", "Wahlbeteiligung", "Winning_Party", "Municipality_Lowercase"
    ))

# --- merging and output ---
# join dfs
merged_df = all_election_df.join(df_spending_agg, on=["Municipality_Lowercase", "Year"], how="inner").drop("Municipality")
# write to csv
merged_df.coalesce(1).write.mode("overwrite").option("header", "true").option("encoding", "latin1").csv("data/merged_data.csv")
print("Merged data sample:")
merged_df.show(5, truncate=False)

Processing data for 2008
Processing data for 2013
Processing data for 2017
Processing data for 2019
Processing data for 2024
Merged data sample:
+----------------------+----+---------------+------------------------------+---------------+------------------+----------------+---------------+-------------+-------------------+
|Municipality_Lowercase|Year|Municipality_ID|Municipality_Name             |Wahlberechtigte|abgegebene_Stimmen|gueltige_Stimmen|Wahlbeteiligung|Winning_Party|Spending_Summe     |
+----------------------+----+---------------+------------------------------+---------------+------------------+----------------+---------------+-------------+-------------------+
|linz                  |2008|40101          |Linz                          |142125         |96209             |94496           |67.69          |SPO          |1.203534975E8      |
|steyr                 |2008|40201          |Steyr                         |28962          |20765             |20335           |71.7       